In [6]:
import torch
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from torchvision.models.resnet import ResNet50_Weights
from torchvision.models.densenet import DenseNet161_Weights
from torchvision.models.resnet import ResNeXt50_32X4D_Weights
from torchvision.models.resnet import ResNeXt101_64X4D_Weights
from torchvision.models.resnet import ResNet101_Weights
import os
import cv2
import pandas as pd

In [7]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)
#Model Info
# Hyperparameters
LEARNING_RATE = 0.0005
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8

EPOCHS = 20
NUM_WORKERS = 0
PIN_MEMORY = False
TRAIN_IMG_DIR = "/content/drive/MyDrive/ism_project_2023/train"
TRAIN_CSV = "/content/drive/MyDrive/ism_project_2023/train.csv"
NUM_CLASSES = 4

LOAD_MODEL = False

# Augmentation parameters
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

cuda


In [10]:


class ColonCancerDataset(Dataset):
    def __init__(self, image_dir, annotation_file, transform):
        self.image_dir = image_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        self.csv = pd.read_csv(annotation_file)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):  #iterator over the dataset
        """
        index: acts as an iterator over the dataset

        return:
        image: torch tensor of format [batch_size, height, width, channels]
        label: torch tensor of integer type of format [batch_size, label_value]
        """

        image_path = os.path.join(self.image_dir, self.images[index])
        #print("image_path: ", image_path)

        #read image and labels
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image_basename = os.path.splitext(os.path.basename(image_path))[0] #without '.jpg'
        #print("image_basename: ", image_basename)

        #check the basename present in the name column in csv file and get the corresponding label value
        label = self.csv[self.csv['name'] == image_basename]['label'].values[0]
        #print("label: ", label)

        #applying augmentations
        if self.transform:
            augmentations = self.transform(image=image)
            image = augmentations['image']

        return image, label





In [11]:
class Train():
  def __init__(self) -> None:
      print("Inside Training Loop")

  def train(self,trainloader, model, optimizer, loss_fn, DEVICE='cuda'):
    loop = tqdm(trainloader, leave=True)
    running_loss = 0.0
    model.train()

    for batch_idx, (image, targets) in enumerate(loop):
        image = image.to(DEVICE) #to GPU
        targets = targets.to(DEVICE)
        #print("data.shape: ", image.shape)
        #print("targets.shape: ", targets.shape)
        #time.sleep(10)

        optimizer.zero_grad()
        predictions = model(image)
        loss = loss_fn(predictions, targets)
        loss.backward()
        optimizer.step()
        loop.set_postfix(loss=loss.item()) #loss over items in a single batch
        running_loss += loss.item() #loss over all batches

    train_loss = running_loss / len(trainloader)
    return train_loss

  def validation(self,validloader, model, loss_fn, DEVICE='cuda'):
      loop = tqdm(validloader, leave=True)
      model.eval()
      valid_loss = 0.0

      with torch.no_grad():
          for batch_idx, (image, targets) in enumerate(loop):
              image = image.to(DEVICE)
              targets = targets.to(DEVICE)

              preds = model(image)
              loss = loss_fn(preds, targets)
              loop.set_postfix(loss=loss.item())
              valid_loss += loss.item()

      valid_loss = valid_loss / len(validloader)
      return valid_loss

In [12]:

  #augmentations
  transform = A.Compose(
      [
          A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH), #always use resizing to reduce computation
          A.HorizontalFlip(p=0.5),
          A.VerticalFlip(p=0.2),
          A.Rotate(limit=[-30,30]),
          A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #using imagenet mean and std
          ToTensorV2(), #converting to the pytorch required format: [batch, channels, height, width]
      ]
  )

  if transform is not None:
      print("INFO: Augmentations applied")
    #load the images and labels
  dataset = ColonCancerDataset(
      image_dir=TRAIN_IMG_DIR,
      annotation_file=TRAIN_CSV,
      transform=transform,
  )

  print("Dataset loaded")

  #split: 80% train, 20% validation
  train_size = int(0.6 * len(dataset))
  valid_size = len(dataset) - train_size
  trainset, validset = torch.utils.data.random_split(dataset, [train_size, valid_size])
  print(f"INFO: Training data split, TRAIN: {train_size}, VALID: {valid_size}")

  # Further split validation set into validation and test sets
  valid_size = int(0.5 * len(validset))
  test_size = len(validset) - valid_size
  valid_dataset, test_dataset = torch.utils.data.random_split(validset, [valid_size, test_size])
  print(f"INFO: Training data split, TRAIN: {train_size}, VALID: {valid_size},TEST: {test_size}")

  #dataloaders
  train_loader = DataLoader(
      dataset=trainset,
      batch_size=TRAIN_BATCH_SIZE,
      num_workers=NUM_WORKERS,
      pin_memory=PIN_MEMORY,
      shuffle=True,
  )

  valid_loader = DataLoader(
      dataset=validset,
      batch_size=VALID_BATCH_SIZE,
      num_workers=NUM_WORKERS,
      pin_memory=PIN_MEMORY,
      shuffle=False,
  )

  test_loader = DataLoader(
      dataset=test_dataset,
      batch_size=TEST_BATCH_SIZE,
      num_workers=NUM_WORKERS,
      pin_memory=PIN_MEMORY,
      shuffle=False,
  )

  print("Train, Val, Test Loaded")

  #loading model and setting hyperparameters

  model = models.resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)

  model.fc = torch.nn.Linear(in_features=2048, out_features=NUM_CLASSES) #changing last layer
  model.to(DEVICE)

  optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
  loss_fn = torch.nn.CrossEntropyLoss()

  best_loss = 0.90
  best_epoch = 0

  print(model)
  utils=Train()
  print("INFO: Training started")
  for epoch in range(1, EPOCHS+1):
      print(f"Epoch: {epoch}/{EPOCHS}")
      train_loss = utils.train(train_loader, model, optimizer, loss_fn)
      valid_loss = utils.validation(valid_loader, model, loss_fn)

      print(f"INFO: Training loss: {train_loss:.3f}")
      print(f"INFO: Validation loss {valid_loss:.3f}")

      #saving best model based on losses
      if valid_loss < best_loss:
          best_loss = valid_loss
          torch.save(model.state_dict(), "best_resnext50_0.005_model.pth")
          best_epoch = epoch
          print(f"Best model saved at epoch {best_epoch} with loss: {best_loss:.3f}")

  print("INFO: Training completed")

  #test_model

  print("Testing Started!")
  model.load_state_dict(torch.load("best_resnext50_model.pth"))
  model.to(DEVICE)
  model.eval()
  from sklearn.metrics import accuracy_score, classification_report

  test_predictions = []
  test_labels = []

  with torch.no_grad():
      for inputs, labels in test_loader:
          inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
          outputs = model(inputs)
          _, predicted = torch.max(outputs, 1)

          test_predictions.extend(predicted.cpu().numpy())
          test_labels.extend(labels.cpu().numpy())

  accuracy = accuracy_score(test_labels, test_predictions)
  classification_report_result = classification_report(test_labels, test_predictions)

  print(f"Accuracy: {accuracy:.4f}")
  print("Classification Report:")
  print(classification_report_result)


INFO: Augmentations applied
Dataset loaded
INFO: Training data split, TRAIN: 2412, VALID: 1609
INFO: Training data split, TRAIN: 2412, VALID: 804,TEST: 805
Train, Val, Test Loaded
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm

100%|██████████| 202/202 [00:33<00:00,  6.02it/s, loss=1.45]


INFO: Training loss: 0.987
INFO: Validation loss 0.975
Epoch: 2/20


100%|██████████| 202/202 [00:32<00:00,  6.16it/s, loss=1.12]


INFO: Training loss: 0.852
INFO: Validation loss 0.695
Best model saved at epoch 2 with loss: 0.695
Epoch: 3/20


...

INFO: Training loss: 0.581
INFO: Validation loss 0.644
Epoch: 19/20


100%|██████████| 202/202 [00:31<00:00,  6.36it/s, loss=1.86]


INFO: Training loss: 0.539
INFO: Validation loss 0.592
Best model saved at epoch 19 with loss: 0.592
Epoch: 20/20


100%|██████████| 202/202 [00:31<00:00,  6.40it/s, loss=2.32]


INFO: Training loss: 0.538
INFO: Validation loss 0.658
INFO: Training completed
Testing Started!
Accuracy: 0.8807
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.89      0.90       218
           1       0.72      0.85      0.78       176
           2       0.98      0.96      0.97       288
           3       0.88      0.73      0.80       123

    accuracy                           0.88       805
   macro avg       0.87      0.86      0.86       805
weighted avg       0.89      0.88      0.88       805

